In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import gensim
import csv, string, os

In [ ]:
# parameters config
flags = tf.flags
FLAGS = flags.FLAGS

tf.flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string("raw_train_data_path", "../raw_data/train.csv", "raw train data path")
flags.DEFINE_string("raw_test_data_path", "../raw_data/test.csv", "raw test data path")
flags.DEFINE_string("deal_train_data_path", "../train_data/deal_train_data.csv", "dealed train data path")
flags.DEFINE_string("train_data_path", "../train_data/train.csv", "train data path")
flags.DEFINE_string("dev_data_path", "../train_data/dev.csv", "dev data path")
flags.DEFINE_string("train_tfrecord_path", "../train_data/train_word_id.tf_record", "train data path")
flags.DEFINE_string("dev_tfrecord_path", "../train_data/dev_word_id.tf_record", "dev data path")
flags.DEFINE_integer("n_hidden", 128, "LSTM hidden layer num of features")
flags.DEFINE_integer("num_step", 16, "input data timesteps")
flags.DEFINE_integer("n_classes", 2, "number of classes")
flags.DEFINE_float("learning_rate", 0.01, "learnning rate")
flags.DEFINE_integer("batch_size", 32, "batch size")
flags.DEFINE_integer("max_steps", 4000, "max step,stop condition")
flags.DEFINE_integer("display_step", 1000, "save model steps")
flags.DEFINE_string("train_writer_path", "./logs/train", "train tensorboard save path")
flags.DEFINE_string("dev_writer_path", "./logs/train", "dev tensorboard save path")
flags.DEFINE_string("checkpoint_path", "./logs/checkpoint", "model save path")
flags.DEFINE_string("glove_path", "./glove.840B.300d/glove.840B.300d.txt", "pre-train embedding model path")
flags.DEFINE_string("gensim_path", "./glove.840B.300d/glove_model.txt", "pre-train embedding model path for gensim")
flags.DEFINE_string("vocab_path", "../train_data/vocab.txt", "pre-train embedding model path")
flags.DEFINE_integer("embedding_dim", 300, "word embedding dim")
flags.DEFINE_integer("seq_length", 15, "sentence max length")

In [ ]:
# read csv
def _read_csv(input_file):
    """
    read csv file,get data
    :param input_file:
    :return:
    """
    with tf.gfile.Open(input_file, "r") as f:
        reader = csv.reader(f)
        lines = []
        for line in reader:
            lines.append(line)
        return lines[1:]  # remove header

In [ ]:
# re-build data file
train_data=pd.read_csv(FLAGS.raw_train_data_path)
target_0_data=train_data.loc[train_data.target==0,:]
target_1_data=train_data.loc[train_data.target==1,:]
# view different target count
print("target=0:%s" % len(target_0_data),"target=1:%s" % len(target_1_data))
# 打乱数据集
target_0_data=target_0_data.sample(frac=1.0)
target_1_data=target_1_data.sample(frac=1.0)
# 切分数据集
target_0_train,target_0_test=target_0_data.iloc[:80000],target_0_data.iloc[80000:]
target_1_train,target_1_test=target_1_data.iloc[:80000],target_1_data.iloc[80000:]
# 合并训练数据并保存
deal_train_data=target_0_train.append(target_1_train)
deal_train_data=deal_train_data.sample(frac=1.0)
deal_train_data.to_csv(FLAGS.deal_train_data_path,index=False)
# build train data
random_all_train_data=deal_train_data.sample(frac=1.0)
# 13w for train and 3w for dev
train_data,dev_data=random_all_train_data.iloc[:130000],random_all_train_data.iloc[130000:]
train_data.to_csv(FLAGS.train_data_path,index=False)
dev_data.to_csv(FLAGS.dev_data_path,index=False)

In [ ]:
# change glove to gensim model
def build_embedding_model(glove_file, gensim_file):
    with open(glove_file, 'r', encoding='utf-8') as f:
        num_lines = 0
        for line in f:
            num_lines += 1
    dims = 300
    gensim_first_line = "{} {}".format(num_lines, dims)
    with open(glove_file, 'r', encoding='utf-8') as fin:
        with open(gensim_file, 'w', encoding='utf-8') as fout:
            fout.write(gensim_first_line + '\n')
            for line in fin:
                fout.write(line)

def sentence_split(sentence, max_length):
    """
    remove punctuation and split sentence.return list of words
    :param sentence:
    :return:
    """
    # sentence = re.sub("[\s+\.\!\/_,$%^*(+\"\')]+|[+——()?【】“”！，。？、~@#￥%……&*（）]+'", "", sentence)
    sentence = [x for x in sentence if x not in string.punctuation]
    sentence = ''.join(sentence)
    words = sentence.split()
    if max_length == 0:
        return words
    else:
        if len(words) > max_length:
            words = words[:max_length]
        elif len(words) < max_length:
            words = words + [" "] * (max_length - len(words))
        return words

# build vocab
def build_vocab(model_file, data_file, vocab_path):
    # load glove model
    model = gensim.models.KeyedVectors.load_word2vec_format(model_file)
    lines = _read_csv(data_file)
    vocab = []
    for line in lines:
        vocab.extend(sentence_split(line[1], 0))
    vocab = set(vocab)
    with open(vocab_path, 'w', encoding='utf-8') as f:
        for word in vocab:
            if word in model:
                f.write(word + ' ' + ' '.join([str(x) for x in model[word]]) + '\n')
             

build_embedding_model(FLAGS.glove_path, FLAGS.gensim_file)
build_vocab(FLAGS.gensim_file, FLAGS.deal_train_data_path, FLAGS.vocab_path)

In [ ]:
# save data to tf-record
def save_word_ids(save_path, csv_path, glove_path, embedding_dim, seq_length, mode='train'):
    vocab, embd = loadGloVe(glove_path, embedding_dim)
    # init vocab processor
    vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(seq_length)
    # fit the vocab from glove
    pretrain = vocab_processor.fit(vocab)
    lines = _read_csv(csv_path)
    split_lines = []
    label_list = []
    qid_list = []
    if mode == 'test':
        for line in lines:
            split_lines.append(' '.join(sentence_split(line[1], seq_length)))
            qid_list.append(str.encode(line[0]))
    else:
        for line in lines:
            split_lines.append(' '.join(sentence_split(line[1], seq_length)))
            label_list.append(int(line[2]))
            qid_list.append(str.encode(line[0]))
    word_ids = list(vocab_processor.transform(np.array(split_lines)))

    writer = tf.python_io.TFRecordWriter(save_path)

    if mode == 'test':
        for index, line in enumerate(word_ids):
            example = tf.train.Example(features=tf.train.Features(feature={
                "qid":
                    tf.train.Feature(bytes_list=tf.train.BytesList(value=[qid_list[index]])),
                "features":
                    tf.train.Feature(int64_list=tf.train.Int64List(value=line))
            }))
            writer.write(example.SerializeToString())
    else:
        for index, line in enumerate(word_ids):
            example = tf.train.Example(features=tf.train.Features(feature={
                "qid":
                    tf.train.Feature(bytes_list=tf.train.BytesList(value=[qid_list[index]])),
                "label":
                    tf.train.Feature(int64_list=tf.train.Int64List(value=[label_list[index]])),
                "features":
                    tf.train.Feature(int64_list=tf.train.Int64List(value=line))
            }))
            writer.write(example.SerializeToString())
    writer.close()
    

save_word_ids(FLAGS.dev_tfrecord_path, FLAGS.dev_data_path, FLAGS.vocab_path, FLAGS.embedding_dim, FLAGS.seq_length)
save_word_ids(FLAGS.train_tfrecord_path, FLAGS.train_data_path, FLAGS.vocab_path, FLAGS.embedding_dim, FLAGS.seq_length)

In [ ]:
# bi-lstm model
class bi_lstm():
    def __init__(self):
        pass

    def model(self, n_hidden, input_data, weights, biases):
        lstm_fw_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
        lstm_fw_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_fw_cell, output_keep_prob=0.7)
        lstm_bw_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
        lstm_bw_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_bw_cell, output_keep_prob=0.7)

        outputs, _ = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, input_data, dtype=tf.float32)
        # 双向LSTM，输出outputs为两个cell的output
        # 将两个cell的outputs进行拼接
        outputs = tf.concat(outputs, 2)
        return tf.matmul(tf.transpose(outputs, [1, 0, 2])[-1], weights['out']) + biases['out']

In [ ]:
def loadGloVe(filename, emb_size):
    vocab = []
    embd = []
    vocab.append('unk')  # 装载不认识的词
    embd.append([0] * emb_size)  # 这个emb_size可能需要指定
    file = open(filename, 'r', encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append(row[1:])
    print('Loaded GloVe!')
    file.close()
    return vocab, embd

def build_embedding_layer(vocab, embd):
    vocab_size = len(vocab)
    embedding_dim = len(embd[0])
    embedding = np.asarray(embd)
    W = tf.Variable(tf.constant(0.0, shape=[vocab_size, embedding_dim]), trainable=False, name="W")
    embedding_placeholder = tf.placeholder(tf.float32, [vocab_size, embedding_dim])
    embedding_init = W.assign(embedding_placeholder)
    return embedding_init, embedding, W, embedding_placeholder, vocab_size

def read_from_tfrecords(tfrecord_dir, batch_size, max_length, n_classes, epochs, mode='train'):
    """
    read data from tf_records
    :param tfrecord_dir:
    :return:
    """
    # build file queue
    file_queue = tf.train.string_input_producer([tfrecord_dir], num_epochs=epochs)
    # build reader
    reader = tf.TFRecordReader()
    _, value = reader.read(file_queue)
    if mode == 'test':
        features = tf.parse_single_example(value, features={
            "qid": tf.FixedLenFeature([1], tf.string),
            "features": tf.FixedLenFeature([max_length], tf.int64)
        })
        qid =features["qid"]
        vector = features["features"]
        vector_batch, qid_batch = tf.train.batch([vector, qid], batch_size=batch_size, num_threads=4, capacity=256)
        return vector_batch, qid_batch
    else:
        features = tf.parse_single_example(value, features={
            "qid": tf.FixedLenFeature([1], tf.string),
            "features": tf.FixedLenFeature([max_length], tf.int64),
            "label": tf.FixedLenFeature([1], tf.int64)
        })

        label = tf.cast(features["label"], tf.int32)  # tf.cast(features["label"], tf.string)
        vector = features["features"]

        vector_batch, label_batch = tf.train.batch([vector, label], batch_size=batch_size, num_threads=4, capacity=256)

        # deal with label batch, change int label to one-hot code
        indices = tf.expand_dims(tf.range(0, batch_size, 1), 1)
        concated = tf.concat([indices, label_batch], 1)
        onehot_labels = tf.sparse_to_dense(concated, tf.stack([batch_size, n_classes]), 1.0, 0.0)

        return vector_batch, onehot_labels

In [ ]:
# train
# tensorflow graph input
input_data = tf.placeholder(dtype=tf.float32, shape=[None, FLAGS.seq_length, FLAGS.embedding_dim], name='input_data')

y = tf.placeholder("float", [None, FLAGS.n_classes])

# get data batch
x_train_batch, y_train_batch = read_from_tfrecords(FLAGS.train_tfrecord_path, FLAGS.batch_size, FLAGS.seq_length,
                                                   FLAGS.n_classes, 2)
x_test, y_test = read_from_tfrecords(FLAGS.dev_tfrecord_path, FLAGS.batch_size, FLAGS.seq_length,
                                     FLAGS.n_classes, 2)

# Define weights
weights = {
    # Hidden layer weights => 2*n_hidden because of foward + backward cells
    'out': tf.Variable(tf.random_normal([2 * FLAGS.n_hidden, FLAGS.n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([FLAGS.n_classes]))
}

pred = bi_lstm().model(FLAGS.n_hidden, input_data, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate).minimize(cost)

# Evaluate model
y_pred_cls = tf.argmax(tf.nn.softmax(pred), 1)
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# tensorboard
tf.summary.scalar('loss', cost)
tf.summary.scalar('accuracy', accuracy)
merged_summary = tf.summary.merge_all()

# Initializing the variables
init = tf.group(tf.initialize_all_variables(), tf.local_variables_initializer())  # tf.global_variables_initializer()

tf.logging.info('Start Training...')

saver = tf.train.Saver()

with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(FLAGS.train_writer_path, sess.graph)
    dev_writer = tf.summary.FileWriter(FLAGS.dev_writer_path, sess.graph)

    sess.run(init)

    step = 1
    tf.logging.info("into embedding layer")
    # embedding layer
    vocab, embd = loadGloVe(FLAGS.glove_path, FLAGS.embedding_dim)
    embedding_init, embedding, W, embedding_placeholder, vocab_size = build_embedding_layer(vocab, embd)
    W = sess.run(embedding_init, feed_dict={embedding_placeholder: embedding})
    # embedding text
    x_train_batch = tf.nn.embedding_lookup(W, x_train_batch, name='train_text_embedding')
    x_test = tf.nn.embedding_lookup(W, x_test, name='test_text_embedding')

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    try:
        while not coord.should_stop():
            curr_x_train_batch, curr_y_train_batch = sess.run([x_train_batch, y_train_batch])
            # tf.logging.info("start %s step optimizer" % step)
            sess.run(optimizer, feed_dict={
                input_data: curr_x_train_batch,
                y: curr_y_train_batch
            })
            if step % FLAGS.every_step == 0:
                # Calculate batch accuracy and loss
                acc, loss, summary = sess.run([accuracy, cost, merged_summary],
                                              feed_dict={input_data: curr_x_train_batch, y: curr_y_train_batch})

                tf.logging.info("Iter " + str(step) + ", Minibatch Loss= " + \
                                "{:.6f}".format(loss) + ", Training Accuracy= " + \
                                "{:.5f}".format(acc))
                train_writer.add_summary(summary, step)
            if step % FLAGS.display_step == 0 or coord.should_stop():
                curr_x_test_batch, curr_y_test_batch = sess.run([x_test, y_test])  # shape(32,15)
                # Calculate test batch accuracy and prediction value
                test_acc, pre, test_summary = sess.run([accuracy, y_pred_cls, merged_summary],
                                                       feed_dict={input_data: curr_x_test_batch, y: curr_y_test_batch})
                print("Step:%s ,Testing Accuracy:" % step, test_acc)
                # save model
                saver.save(sess, FLAGS.checkpoint_path + '/model-%s' % step, global_step=step)
                # get real value
                y_true = curr_y_test_batch[:, 1]
                # calculate evaluate value
                # tf_p, tf_r, tf_f1 = sess.run(calculate_evaluate_value(pre, y_true))
                # print("prediction:%s   recall:%s   f1_score:%s" % (tf_p, tf_r, tf_f1))
                # 
                # # evaluate by sklearn
                # # 评估
                # print("Precision, Recall and F1-Score...")
                # print(metrics.classification_report(y_true, pre, target_names=['无意义', '有意义']))
                # 
                # # 混淆矩阵
                # print("Confusion Matrix...")
                # cm = metrics.confusion_matrix(y_true, pre)
                # print(cm)

                with tf.name_scope('Evaluation'):
                    tf.summary.scalar('prediction', tf_p)
                    tf.summary.scalar('recall', tf_r)
                    tf.summary.scalar('f1_score', tf_f1)
                dev_writer.add_summary(test_summary, step)

            step += 1
    except tf.errors.OutOfRangeError:
        print('Done training -- epoch limit reached')
    finally:
        tf.logging.info("Optimization Finished!")
        coord.request_stop()
    coord.join(threads)

In [ ]:
# build test data tf_record
if os.path.exists(FLAGS.test_tfrecord_path) is not True:
    save_word_ids(FLAGS.test_tfrecord_path, FLAGS.test_data_path, FLAGS.glove_path, FLAGS.embedding_dim,
                  FLAGS.seq_length, 'test')

with tf.Graph().as_default():
    # tensorflow graph input
    input_data = tf.placeholder(dtype=tf.float32, shape=[None, FLAGS.seq_length, FLAGS.embedding_dim],
                                name='input_data')
    # get test data
    test_text_batch, test_qid_batch = read_from_tfrecords(FLAGS.test_tfrecord_path, FLAGS.batch_size, FLAGS.seq_length,
                                                          FLAGS.n_classes, 1, 'test')
    # ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint_path)  # 通过检查点文件锁定最新的模型
    # saver = tf.train.import_meta_graph(ckpt.model_checkpoint_path + '.meta')  # 载入图结构，保存在.meta文件中
    # Define weights
    weights = {
        # Hidden layer weights => 2*n_hidden because of foward + backward cells
        'out': tf.Variable(tf.random_normal([2 * FLAGS.n_hidden, FLAGS.n_classes]))
    }
    biases = {
        'out': tf.Variable(tf.random_normal([FLAGS.n_classes]))
    }
    pred = bi_lstm().model(FLAGS.n_hidden, input_data, weights, biases)
    logit = tf.argmax(tf.nn.softmax(pred), 1)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        # sess.run(tf.initialize_all_variables())
        sess.run(tf.local_variables_initializer())

        # embedding layer
        vocab, embd = loadGloVe(FLAGS.glove_path, FLAGS.embedding_dim)
        embedding_init, embedding, W, embedding_placeholder, vocab_size = build_embedding_layer(vocab, embd)
        W = sess.run(embedding_init, feed_dict={embedding_placeholder: embedding})
        # embedding text
        test_text_batch = tf.nn.embedding_lookup(W, test_text_batch, name='train_text_embedding')

        ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint_path)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)  # 载入参数，参数保存在两个文件中，不过restore会自己寻找
        else:
            tf.logging.ERROR("Load model failed, can't find model")
            raise FileNotFoundError
        # graph = tf.get_default_graph()
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        save_qid_list = []
        save_target_list = []
        try:
            while not coord.should_stop():
                test_batch_value, qid = sess.run([test_text_batch, test_qid_batch])
                pred_value = sess.run(logit, feed_dict={input_data: test_batch_value})
                save_qid_list.extend(qid)
                save_target_list.extend(pred_value)
        except tf.errors.OutOfRangeError:
            print("Done testing -- epoch limit reached")
        finally:
            coord.request_stop()
        coord.join(threads)
        # write result to csv
        headers = ['qid', 'target']
        save_data_list = []
        for index, qid in enumerate(save_qid_list):
            qid_str = qid[0].decode()
            save_data_list.append([qid_str, save_target_list[index]])
        with open(FLAGS.submit_file, 'w+') as f:
            f_csv = csv.writer(f)
            f_csv.writerow(headers)
            f_csv.writerows(save_data_list)